In [2]:
ws = 3
wg = 6
s_12 = 0.6
s_21 = 0.55

In [2]:
"""
score1: Number of points won by server in the game
score2: Number of points won by returner in the game
winning_total: The number of points won by a player to win the game
p: Probability of server winning a point
memo: A helper variable for computational efficiency
"""
def prob_server_wins_game(score1, score2, winning_total, p, memo):
    if score1 == winning_total and score2 <= winning_total - 2:
        return 1
    elif score2 == winning_total and score1 <= winning_total - 2:
        return 0
    elif score1 == winning_total - 1 and score2 == winning_total - 1:
        memo[(score1, score2)] = prob_server_wins_deuce(p)
    else:
        server_wins_point = prob_server_wins_game(score1 + 1, score2, winning_total, p, memo)
        returner_wins_point = prob_server_wins_game(score1, score2 + 1, winning_total, p, memo)
        memo[(score1, score2)] = p * server_wins_point + (1-p) * returner_wins_point
    return memo[(score1, score2)]    

In [3]:
def prob_server_wins_deuce(p):
    return (p * p) / (1- 2*p*(1-p))

In [44]:
"""
p: probability first server wins a point when serving
q: probability second server wins a point when serving
"""
def prob_1st_server_wins_tiebreakDeuce(p, q):
    num = p * (1-q)
    denom = 1 - (p*q + (1-q)*(1-p))
    return num / denom

In [77]:
def prob_server_wins_set(score1, score2, winning_total, p, q, memo, server):
    if score1 >= winning_total and score2 <= score1 - 2:
        return 1
    elif score2 >= winning_total and score1 <= score2 - 2:
        return 0
    elif score1 == winning_total and score2 == winning_total:
        memo[(score1, score2, server)] = prob_1st_server_wins_tiebreak(0, 0, 7, p, q, {}, server)
        return memo[(score1, score2, server)]
    else:
        if server == 1:
            pr_1st_server_wins_game = prob_server_wins_game(0, 0, 4, p, {})
        elif server == -1:
            pr_1st_server_wins_game = 1 - prob_server_wins_game(0, 0, 4, q, {})
        
        first_server_wins_game = prob_server_wins_set(score1 + 1, score2, winning_total, p, q, memo, server*-1)
        first_server_loses_game = prob_server_wins_set(score1, score2 + 1, winning_total, p, q, memo, server*-1)
        
        memo[(score1, score2, server)] = pr_1st_server_wins_game * first_server_wins_game + (1-pr_1st_server_wins_game)*first_server_loses_game
        
        return memo[(score1, score2, server)]
        
    

In [78]:
def prob_1st_server_wins_tiebreak(score1, score2, winning_total, p, q, memo, server):
    if score1 >= winning_total and score2 <= score1 - 2:
        return 1
    elif score2 >= winning_total and score1 <= score2 - 2:
        return 0
    elif score1 == winning_total - 1 and score2 == winning_total - 1:
        if (server == 1):
            memo[(score1, score2, server)] = prob_1st_server_wins_tiebreakDeuce(p, q)
        else:
            memo[(score1, score2, server)] = 1 - prob_1st_server_wins_tiebreakDeuce(q, p)
        return memo[(score1, score2, server)]
    else:
        if (score1 + score2) % 2 == 0:
            next_server = server * -1
        else:
            next_server = server
        fs_wp = prob_1st_server_wins_tiebreak(score1 + 1, score2, winning_total, p, q, memo, next_server)
        fr_wp = prob_1st_server_wins_tiebreak(score1, score2 + 1, winning_total, p, q, memo, next_server)
        
        if server == 1:
            memo[(score1, score2, server)] = p * fs_wp + (1-p) * fr_wp
        else:
            memo[(score1, score2, server)] = (1-q) * fs_wp + q * fr_wp
        
        return memo[(score1, score2, server)]
        
            

In [91]:
"""
wtm: number of sets needed to win the match
wts: number of games needed to win a set
wtg: number of points needed to win a game
wtt: number of points needed to win a tiebreak
server: 1 if p is to serve first, -1 if q is to serve first
p: probability p_1 wins a point when serving
q: probability p_(-1) wins a point when serving
"""
def match_driver(wtm, wts, wtg, wtt, server, p, q):
    def prob_p_wins_match(sets_p, sets_q, games_p, games_q, memo, server):
        if sets_p == wtm:
            return 1
        elif sets_q == wtm:
            return 0
        elif (sets_p, sets_q, games_p, games_q, server) in memo:
            return memo[(sets_p, sets_q, games_p, games_q, server)]
        else:
            if games_p >= wts and games_q <= games_p - 2:
                memo[(sets_p, sets_q, games_p, games_q, server)] = prob_p_wins_match(sets_p + 1, sets_q, 0, 0, memo, server)
            elif games_q >= wts and games_p <= games_q - 2:
                memo[(sets_p, sets_q, games_p, games_q, server)] = prob_p_wins_match(sets_p, sets_q + 1, 0, 0, memo, server)
            elif games_p == wts and games_q == wts:
                if server == 1:
                    p_wins_tiebreak = prob_1st_server_wins_tiebreak(0, 0, wtt, p, q, {}, 1)
                else:
                    p_wins_tiebreak = 1 - prob_1st_server_wins_tiebreak(0, 0, wtt, q, p, {}, 1)
                
                if_p_wins_tiebreak = prob_p_wins_match(sets_p + 1, sets_q, 0, 0, memo, server * -1)
                if_p_loses_tiebreak = prob_p_wins_match(sets_p, sets_q + 1, 0, 0, memo, server * -1)
                
                memo[(sets_p, sets_q, games_p, games_q, server)] = p_wins_tiebreak*if_p_wins_tiebreak + (1-p_wins_tiebreak)*if_p_loses_tiebreak
            
            else:
                if server == 1:
                    p_wins_game = prob_server_wins_game(0, 0, wtg, p, {})
                else:
                    p_wins_game = 1 - prob_server_wins_game(0, 0, wtg, q, {})
                
                if_p_wins_game = prob_p_wins_match(sets_p, sets_q, games_p + 1, games_q, memo, server * -1)
                if_p_loses_game = prob_p_wins_match(sets_p, sets_q, games_p, games_q + 1, memo, server * -1)
                
                memo[(sets_p, sets_q, games_p, games_q, server)] = p_wins_game*if_p_wins_game + (1-p_wins_game)*if_p_loses_game
        
            return memo[(sets_p, sets_q, games_p, games_q, server)]
    return prob_p_wins_match(0, 0, 0, 0, {}, server)

In [12]:
"""
wtm: number of sets needed to win the match
wts: number of games needed to win a set
wtg: number of points needed to win a game
wtt: number of points needed to win a tiebreak
server: 1 if p is to serve first, -1 if q is to serve first
gp: probability p_1 wins a service game
gq: probability p_(-1) wins a service game
p: probability p_1 wins a server (only relevant for tiebreak)
q: probability p_(-1) wins a serve (only relevant for tiebreak)
"""
def match_driver_game_based_tiebreak_serve_based(wtm, wts, wtg, wtt, server, gp, gq, p, q):
    def prob_p_wins_match(sets_p, sets_q, games_p, games_q, memo, server):
        if sets_p == wtm:
            return 1
        elif sets_q == wtm:
            return 0
        elif (sets_p, sets_q, games_p, games_q, server) in memo:
            return memo[(sets_p, sets_q, games_p, games_q, server)]
        else:
            if games_p >= wts and games_q <= games_p - 2:
                memo[(sets_p, sets_q, games_p, games_q, server)] = prob_p_wins_match(sets_p + 1, sets_q, 0, 0, memo, server)
            elif games_q >= wts and games_p <= games_q - 2:
                memo[(sets_p, sets_q, games_p, games_q, server)] = prob_p_wins_match(sets_p, sets_q + 1, 0, 0, memo, server)
            elif games_p == wts and games_q == wts:
                if server == 1:
                    p_wins_tiebreak = prob_1st_server_wins_tiebreak(0, 0, wtt, p, q, {}, 1)
                else:
                    p_wins_tiebreak = 1 - prob_1st_server_wins_tiebreak(0, 0, wtt, q, p, {}, 1)
                
                if_p_wins_tiebreak = prob_p_wins_match(sets_p + 1, sets_q, 0, 0, memo, server * -1)
                if_p_loses_tiebreak = prob_p_wins_match(sets_p, sets_q + 1, 0, 0, memo, server * -1)
                
                memo[(sets_p, sets_q, games_p, games_q, server)] = p_wins_tiebreak*if_p_wins_tiebreak + (1-p_wins_tiebreak)*if_p_loses_tiebreak
            
            else:
                if server == 1:
                    p_wins_game = gp
                else:
                    p_wins_game = 1 - gq
                
                if_p_wins_game = prob_p_wins_match(sets_p, sets_q, games_p + 1, games_q, memo, server * -1)
                if_p_loses_game = prob_p_wins_match(sets_p, sets_q, games_p, games_q + 1, memo, server * -1)
                
                memo[(sets_p, sets_q, games_p, games_q, server)] = p_wins_game*if_p_wins_game + (1-p_wins_game)*if_p_loses_game
        
            return memo[(sets_p, sets_q, games_p, games_q, server)]
    return prob_p_wins_match(0, 0, 0, 0, {}, server)

In [11]:
"""
wtm: number of sets needed to win the match
sp: probability p_1 wins a set
"""
def match_driver_set_based(wtm, sp):
    def prob_p_wins_match(sets_p, sets_q, memo):
        if sets_p == wtm:
            return 1
        elif sets_q == wtm:
            return 0
        elif (sets_p, sets_q) in memo:
            return memo[(sets_p, sets_q)]
        else:
            if_p_wins_set = prob_p_wins_match(sets_p + 1, sets_q, memo)
            if_p_loses_set = prob_p_wins_match(sets_p, sets_q + 1, memo)
            
            memo[(sets_p, sets_q)] = (sp * if_p_wins_set) + (1-sp) * if_p_loses_set
            return memo[(sets_p, sets_q)]
    return prob_p_wins_match(0, 0, {})

In [ ]:
"""
wtm: number of sets needed to win the match
wts: number of games needed to win a set
wtg: number of points needed to win a game
wtt: number of points needed to win a tiebreak
server: 1 if p is to serve first, -1 if q is to serve first
gp: probability p_1 wins a service game
gq: probability p_(-1) wins a service game
tp: probability p_1 wins a tiebreak
"""
def match_driver_game_based(wtm, wts, wtg, wtt, server, gp, gq, tp):
    def prob_p_wins_match(sets_p, sets_q, games_p, games_q, memo, server):
        if sets_p == wtm:
            return 1
        elif sets_q == wtm:
            return 0
        elif (sets_p, sets_q, games_p, games_q, server) in memo:
            return memo[(sets_p, sets_q, games_p, games_q, server)]
        else:
            if games_p >= wts and games_q <= games_p - 2:
                memo[(sets_p, sets_q, games_p, games_q, server)] = prob_p_wins_match(sets_p + 1, sets_q, 0, 0, memo, server)
            elif games_q >= wts and games_p <= games_q - 2:
                memo[(sets_p, sets_q, games_p, games_q, server)] = prob_p_wins_match(sets_p, sets_q + 1, 0, 0, memo, server)
            elif games_p == wts and games_q == wts:
                if server == 1:
                    p_wins_tiebreak = tp
                else:
                    p_wins_tiebreak = 1 - tp
                
                if_p_wins_tiebreak = prob_p_wins_match(sets_p + 1, sets_q, 0, 0, memo, server * -1)
                if_p_loses_tiebreak = prob_p_wins_match(sets_p, sets_q + 1, 0, 0, memo, server * -1)
                
                memo[(sets_p, sets_q, games_p, games_q, server)] = p_wins_tiebreak*if_p_wins_tiebreak + (1-p_wins_tiebreak)*if_p_loses_tiebreak
            
            else:
                if server == 1:
                    p_wins_game = gp
                else:
                    p_wins_game = 1 - gq
                
                if_p_wins_game = prob_p_wins_match(sets_p, sets_q, games_p + 1, games_q, memo, server * -1)
                if_p_loses_game = prob_p_wins_match(sets_p, sets_q, games_p, games_q + 1, memo, server * -1)
                
                memo[(sets_p, sets_q, games_p, games_q, server)] = p_wins_game*if_p_wins_game + (1-p_wins_game)*if_p_loses_game
        
            return memo[(sets_p, sets_q, games_p, games_q, server)]
    return prob_p_wins_match(0, 0, 0, 0, {}, server)

In [106]:
wtm = 3
wts = 6
wtg = 4
wtt = 7
server = -1
p = 0.5
q = 0.51

match_driver(wtm, wts, wtg, wtt, server, p, q)

0.4332352501977097

In [4]:
prob_server_wins_game(0, 0, 4, 0.683237, {})

0.8795501924897635

In [58]:
import random
def tiebreak_deuce_simulator(score1, score2, p, q, server):
    if score1 >= score2 + 2:
        return 1
    elif score2 >= score1 + 2:
        return -1
    else:
        point = random.random()
        if server == 1:
            if point <= p:
                score1 += 1
            else:
                score2 += 1
        elif server == -1:
            if point <= q:
                score2 += 1
            else:
                score1 += 1
        if (score1 + score2) % 2 == 1:
            return tiebreak_deuce_simulator(score1, score2, p, q, server * -1)
        else:
            return tiebreak_deuce_simulator(score1, score2, p, q, server)

N = 100000
c = 0
p = 0.5
q = 0.3
server = 1
for i in range(N):
    turn = tiebreak_deuce_simulator(0, 0, p, q, server)
    if turn == 1:
        c += 1
print(c/N)

0.70186


In [65]:
import random
def tiebreak_simulator(score1, score2, winning_total, p, q, server):
    if score1 >= winning_total and score2 <= score1 - 2:
        return 1
    elif score2 >= winning_total and score1 <= score2 - 2:
        return -1
    elif score1 == winning_total - 1 and score2 == winning_total - 1:
        return tiebreak_deuce_simulator(score1, score2, p, q, server)
    else:
        point = random.random()
        if server == 1:
            if point <= p:
                score1 += 1
            else:
                score2 += 1
        elif server == -1:
            if point <= q:
                score2 += 1
            else:
                score1 += 1
        if (score1 + score2) % 2 == 1:
            return tiebreak_simulator(score1, score2, winning_total, p, q, server * -1)
        else:
            return tiebreak_simulator(score1, score2, winning_total, p, q, server)
N = 1000000
c = 0
p = 0.9
q = 0.6
server = 1
for i in range(N):
    turn = tiebreak_simulator(0, 0, 7, p, q, server)
    if turn == 1:
        c += 1
print(c/N)

0.933518
